# API's in Data Science

## What is an API? 

- API stands for **Application Programming Interface**.

- A set of rules and protocols that allows different software applications to communicate with each other.

- In the context of Python, APIs are used to request and receive data from web servers.

- Specifically a **REST API** is a type of online service that we can access to receiver (and send) data.

## Why use an API?

- Efficient and structured way to access data

- Less prone to breaking than web scraping - data is served in standard formats like JSON.
  
- Allows third-party interaction with other services e.g social media, weather apps etc

## The Data Restaurant 

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcReqcxhovo8LV1uRKmOZ1TNnbCNrsCU5sJ7Cg&s">

Imagine you're in a restaurant...

- You **(the client/customer)**
- The waiter **(API)**
- The kitchen **(Data Source)**

Today you're hungry for data! 

To get your data food, your order needs to get to the kitchen, so like in any restaurant, you make a request via a waiter. 

Once your meal (data) has been prepared by the kitchen, the waiter will bring it to your table. 

This is a great analogy for understanding what API's do: they take requests for data, collect data from a server, and then deliver the response back to the user.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*re837GMp63hzDnB8tEiZKA.png" style="width: 100%">

*If you wanted a comparison analogy for webscraping, that would be like going to a buffet and picking individual food items to make a meal.*

## HTTP Requests  

-  HTTP Requests are what we will use to query an API for data. 
  
-  We point to a specific URL, and give some extra information about the type of action we are doing (e.g. getting data, sending data, deleting data) 
  
-  We can give some additional parameters (e.g. authorisation, specific queries etc...)
  
- We then get a response that tells us whether our request was successful and hopefully, the information we requested.

#### There are common request types:

-  **GET: Retrieve data from the server.**
-  
-  **POST: Submit data to the server.**
-  
-  **PUT: Update existing data on the server.**
-  
-  **DELETE: Remove data from the server.**


We'll be focusing on **GET** requests.



## Practical Considerations of Working with API's 

### Authentication 

- Many APIs require an API key or OAuth for access.

- Always keep your keys secure and **never hard-code them in your scripts!**

### Rate Limiting 

- APIs often have rate limits to control the number of requests you can make.

### Data Formats 

- JSON is the most common format for API responses.


### HTTP Request Status Codes

- ```200``` Successful response! 
- ```400``` Client Error - there is something wrong your side, check authorisation and request.
- ```500``` Server Error - there is something wrong with the server, try again later. 

# Collecting data from API's with python for Data Science
<img src="https://cdn.sanity.io/images/oaglaatp/production/6fdc068d96ae659dac4248b1336fa9ce79b90cef-850x254.png?w=675&h=202&fit=crop" style="width: 100%">

When collecting data from API's for data science purposes we'll be following this process: 
1. Review the API documentation 
   
2. Request data from the API.
   
3. Analyse the JSON response and select the data we want. 
   
4. Store the data in a pandas data frame for cleaning and analysis. 
   
We'll also look at exporting this as CSV and Excel files. 

## Python libraries 

To make HTTP requests we'll be using the ```requests``` library. 

https://realpython.com/python-requests/

# Example: YouTube API

Let's have a look at a code example...

In [ ]:
! pip install requests

In [ ]:
import requests 
import pandas as pd 
import json
from api_keys import YOUTUBE_KEY

## Test the requests library

In [ ]:
response = requests.get("https://api.github.com")

print(response.status_code)

response.json()

## Working with the YouTube API

The hardest part of working with API's is figuring out the structure of the URL.

We currently have the root URL:

In [ ]:
url = "https://www.googleapis.com/youtube/v3/"

This is the location of our data. Now we need to define what data we want to collect.

In this example we want to get video ID and information about the video.

To get this right we need to look at the YouTube API documentation:

<img src="https://cdn.sanity.io/images/oaglaatp/production/0431c1685464b66074c67a6307449eb4825b7e28-919x665.png?w=675&h=488&fit=crop">

In [ ]:
api_key = YOUTUBE_KEY
channel_id = "UC8butISFwT-Wl7EV0hUK0BQ" # freecodecamps channel id
page_token = ""

request_url = f"{url}search?key={api_key}&channelId={channel_id}&part=snippet,id&order=date&maxResults=1000&pageToken={page_token}"
request_url

Here we are performing a search through the YouTube API. Everything to the right of ```?``` is the parameters which are each separated by ```&```. 

We've used a f string to combine variables into one string.

- First is the base URL
- Then our API key
- Then the channel we want to search 
- Then part, where we specify we want snippet and id data. 
- Then order, which is by date.
- Then, max results is set to 1000.
- Final pageToken, which we'll look at later...

It can be pretty frustrating trying build the url for your use case. So start from the base url and work your way to the data you want. 

## Requesting data

Now we've got our URL we can use it to GET the data we want.

In [ ]:
response = requests.get(request_url)

response.status_code

We got a 200 status code, which means the request was successful! 

Now let's save this data in a JSON object and take a look.

In [ ]:
response_json = response.json()

response_json

## Filtering the Data 
Great! We've made an API call and collected data, but we don't need all of it.

Looking at the JSON data we can see where each video starts and ends, and the key value pairs in each video.

Let's isolate the first video in the data and some specific information. 

In [ ]:
first_video = response_json['items'][0]

first_video


**Let's get the video id, title, and upload date and store them in variables...**

In [ ]:
video_id = first_video['id']['videoId']
video_id

In [ ]:
video_title = first_video['snippet']['title']
video_title

In [ ]:
upload_date = first_video['snippet']['publishedAt']
upload_date

In [ ]:
# we can clean this up and split the date and time by the "T" (time) character
upload_date = str(upload_date).split("T")[0]
upload_date

In [ ]:
video_info = f'Video Title: {video_title}, Video ID: {video_id}, Upload Date: {upload_date}'
video_info

Great, now we have the video id, title, and upload date for the first video in the response. Let's put this into a pandas DataFrame.

In [ ]:
video_df = pd.DataFrame(columns=["Video Title", "Video ID", "Upload Date"], data=[[video_title, video_id, upload_date]])
video_df

## Looping through the collected JSON data
Ok, we've managed to get data for one video and store it in a data frame. 

But for data analysis we need lot's of data to make comparisons, so let's make a data frame with all of the videos we got from the API call.

In [ ]:
for video in response_json['items']: # loop through the items in the response
    if video['id']['kind'] == 'youtube#video': # check if the item is a video! 
        video_id = video['id']['videoId']  # get the video ID
        video_title = video['snippet']['title'] # get the video title
        upload_date = str(video['snippet']['publishedAt']).split("T")[0] # get the upload date and split the time

        video_df = pd.concat([video_df, pd.DataFrame(data=[[video_title, video_id, upload_date]], columns=["Video Title", "Video ID", "Upload Date"])], ignore_index=True)

video_df

## Collecting more data, with the data we've collected... 

We've now got a dataframe of videos from the freecodecamp channel. But this information doesn't really give us much insight into the videos and how they've performed. 

We can use the video ID's we've collected to make an API call for each video to collect additional information. 

Let's collect the view, comments, likes, and dislikes count for each video. 

In [ ]:
video_df = pd.DataFrame(columns=["Video Title", "Video ID", "Upload Date", "View Count", "Like Count", "Comment Count"]) # empty the dataframe

for video in response_json['items']:
    
    if video['id']['kind'] == 'youtube#video': 
        # What we did before 
        video_id = video['id']['videoId'] 
        video_title = video['snippet']['title'] 
        upload_date = str(video['snippet']['publishedAt']).split("T")[0] 
        
        # Make a request to get the video statistics using the video ID
        url_video_stats = f"{url}videos?key={api_key}&part=statistics&id={video_id}"
        response_video_stats = requests.get(url_video_stats)
        response_video_stats_json = response_video_stats.json()['items'][0]['statistics']
        
        # Get the view count, like count, and comment count, and convert them to integers! 
        view_count = int(response_video_stats_json['viewCount'])
        like_count = int(response_video_stats_json['likeCount'])
        comment_count = int(response_video_stats_json['commentCount'])

        video_df = pd.concat([video_df, pd.DataFrame(data=[[video_title, video_id, upload_date, view_count, like_count, comment_count]], 
                                                     columns=["Video Title", "Video ID", "Upload Date", "View Count", "Like Count", "Comment Count"])],
                                                    ignore_index=True)

video_df

## Organising and plotting

Nice! We've got all our data and put it in a data frame. 

Using skills you've acquired from the last few weeks we can oraganise and visualise this.

First reorganise this by likes, with most popular videos appearing first. 


In [ ]:
sort_video_df = video_df.groupby('Video Title')['Like Count'].sum().reset_index().sort_values(by='Like Count', ascending=False)
sort_video_df.reset_index(drop=True, inplace=True)
sort_video_df.head(10)

Now we can plot the top 10 most popular videos by likes

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(sort_video_df['Video Title'][:10], sort_video_df['Like Count'][:10], color='skyblue')
plt.xticks(rotation=90)
plt.ylabel("Like Count")
plt.title("Top 10 Videos by Like Count")
plt.show()

## Exporting the Data

This is what we've done: 

- read the API documentation 
- requested data from the api 
- select what we want with JSON 
- use the data to collect more
- plot the data 

Now finally we can export this data as a CSV or Excel.

In [ ]:
sort_video_df.to_csv('sorted_videos.csv')

video_df.to_excel('video_data.xlsx')

Let's have a look at another example with <a href="examples/spotipy.ipynb">Spotipy</a>, which is a standalone python library for interacting with the spotify API. 

# Overview

In this session, we covered the following topics related to APIs:

- **Introduction to APIs**: Understanding what APIs are and their basic functionality.
- **Making API Requests**: How to make simple API requests using HTTP with Python requests library.
- **Handling API Responses**: Basic techniques for parsing and handling responses from APIs.
- **Extracting Data from Requests**: Converting JSON data to pandas dataframes and plotting it. 

**Resources:**
- Youtube: https://developers.google.com/youtube/v3/docs
- Spotipy: https://spotipy.readthedocs.io/en/2.24.0/
- Python Requests: https://www.w3schools.com/python/module_requests.asp
- Pandas: https://www.w3schools.com/python/pandas/pandas_cleaning.asp 
- Matplotlib: https://matplotlib.org/
- Python JSON: https://www.w3schools.com/python/python_json.asp

**List of public APIs**: https://github.com/public-apis/public-apis

**Credits**:

This class was made with reference to https://git.arts.ac.uk/mtanska/23-24-bsc-2-big-data/blob/main/week04/week04_coding_demo1.ipynb by Louis McCallum, Terrance Broad, Polo Sologub, and https://www.kdnuggets.com/2021/09/python-apis-data-science-project.html by Nate Rosidi. 

# Before you start the labs!

## Create an openweather account and get your API key.
Sometimes it takes a while to update on their server, so do this *before* we have a break! 
<a href="https://docs.openweather.co.uk/appid">Instructions Here</a>

